In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# LCLS Classic model 

In [2]:
from LCLS.bmad import LCLSTaoModel
from LCLS.epics import epics_proxy

import numpy as np
import os
import json
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Make sure this exists
assert 'LCLS_LATTICE' in os.environ

In [4]:
# Cached EPICS pv data
epics = epics_proxy('data/epics_snapshot_2018-03-06T11:22:45.000000-08:00.json', verbose=True)

Loaded data/epics_snapshot_2018-03-06T11:22:45.000000-08:00.json with 1722 PVs


In [5]:
M = LCLSTaoModel('cu_hxr', epics = epics,verbose=True, ploton=True)
print(M)

Initialized Tao with /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmpkqj0sttf/tao/tao.init
Configured.
_______________________________________________
_______________________________________________
LCLS Copper Linac EPICS info

Bunch charge off cathode:  249.4 pC
VCC offset x, y:   -0.5,    0.5 mm
Laser heater energy      39.35 μJ =>  44.85 keV rms energy spread
Bunch charge in LTU:          0.5069 pC
BC1 mean current:              251.8 A
BC2 peak current:             3667.1 A

Linac    Energy (MeV)      Phase (deg)    fudge (%)
L1       238.862        -27.34        99.71
L1X       220.038        -160.0
L2        5000.1        -32.37        101.6
L3       14429.0          -8.0        103.3

FEL Pulse energy :                2.6 mJ
_______________________________________________


In [6]:
%%tao 
sho var linac_fudge

-------------------------
Tao> sho var linac_fudge
Variable name:  linac_fudge

 Index  Controlled Attributes(s)    Meas         Model        Design  Useit_opt
     1  O_L1_FUDGE[F]           0.0000E+00    1.0000E+00    1.0000E+00       F
     2  O_L2_FUDGE[F]           0.0000E+00    1.0000E+00    1.0000E+00       F
     3  O_L3_FUDGE[F]           0.0000E+00    1.0000E+00    1.0000E+00       F
 Index  Controlled Attributes(s)    Meas         Model        Design  Useit_opt
-------------------------
Tao> 


## Track particles

In [7]:
%%tao
set beam_init beam_track_end = UEEND
set csr_param n_bin = 40
snparticle 10000
beamon
beamoff

-------------------------
Tao> set beam_init beam_track_end = UEEND
-------------------------
Tao> set csr_param n_bin = 40
-------------------------
Tao> snparticle 10000
-------------------------
Tao> beamon
Beam at Element: 1366. Time: 1 min
-------------------------
Tao> beamoff
-------------------------
Tao> 


'/Users/chrisonian/Code/GitHub/lcls-live/examples'

In [10]:
h5file1 = os.path.join(os.getcwd(), 'test1.h5')
M.cmd('write beam -at UEBEG '+h5file1)

h5file2 = os.path.join(os.getcwd(), 'test2.h5')
M.cmd('write beam -at UEEND '+h5file2)

['[INFO] tao_write_cmd:',
 '    Written: /Users/chrisonian/Code/GitHub/lcls-live/examples/test2.h5']

# Plot

In [12]:
from opmd_beamphysics import bunch_plotting, bunch_tools, bunch_stats
from bokeh.plotting import  show, output_notebook
from bokeh.layouts import column, row
output_notebook(verbose=False, hide_banner=True)
from h5py import File

In [14]:
#h1.close()
#h2.close()
h1 = File(h5file1, 'r')
h2 = File(h5file2, 'r')
bunch1 = h1['data/00001/particles']
bunch2 = h2['data/00001/particles']

show(
    row(
        column(    
            bunch_plotting.plot_bunch_h5(bunch1, 't', 'pz', liveOnly=True,  bins = 100),
            bunch_plotting.plot_histogram_h5(bunch1, 't', bins=100)), 
    column(
        
    bunch_plotting.plot_bunch_h5(bunch2, 't', 'pz', liveOnly=True,  bins = 100),
    bunch_plotting.plot_histogram_h5(bunch2, 't', bins=100))        

    )
    
    )

In [15]:
# Cleanup
os.remove(h5file1), os.remove(h5file2)

(None, None)